In [2]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class FaceLandmarksDatasetLimited(Dataset):
    def __init__(self, file_paths, max_sequence_length=100):
        self.file_paths = file_paths
        self.max_sequence_length = max_sequence_length
        self.data = []
        self.labels = []

        for file_path in self.file_paths:
            data = pd.read_csv(file_path, nrows=max_sequence_length)
            label = int(file_path.split('/')[-1].split('_')[0])  # Extract label from file name

            # Remove the timestamp column
            features = data.iloc[:, 1:].values  # Exclude timestamp

            # Normalize the features
            features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

            # Truncate or pad sequences to the max length
            if features.shape[0] > max_sequence_length:
                features = features[:max_sequence_length]
            else:
                padding = np.zeros((max_sequence_length - features.shape[0], features.shape[1]))
                features = np.vstack((features, padding))

            self.data.append(features)
            self.labels.append(label)

        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [1]:
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class FaceLandmarksModelAttention(nn.Module):
    def __init__(self, input_dim=1434, hidden_size=128, num_classes=4, num_heads=4, num_layers=2):
        super(FaceLandmarksModelAttention, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_size)
        self.transformer = TransformerEncoder(
            TransformerEncoderLayer(hidden_size, num_heads, dim_feedforward=256, dropout=0.1),
            num_layers
        )
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)  # Average pooling across sequence length
        x = self.dropout(self.relu(self.fc1(x)))
        out = self.fc2(x)
        return out

In [3]:
import torch
import torch.nn as nn
from sklearn.model_selection import ParameterGrid
from glob import glob
import os

# Define search space
learning_rates = [0.0001, 0.001, 0.01]
batch_sizes = [32, 64, 128]
optimizers = ['Adam', 'SGD', 'RMSprop']
epochs = [200]

search_space = {
    'learning_rate': learning_rates,
    'batch_size': batch_sizes,
    'optimizer': optimizers,
    'epochs': epochs,
}

parameter_grid = ParameterGrid(search_space)

In [4]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset


dataset_folder = "./111/zzz"  # Update with your folder path
csv_files = glob(os.path.join(dataset_folder, "*.csv"))
print(f"Found {len(csv_files)} CSV files.")

# Dataset initialization
dataset = FaceLandmarksDatasetLimited(csv_files)

# Class-wise data splitting
labels = dataset.labels
unique_labels = np.unique(labels)
train_indices, valid_indices = [], []

for label in unique_labels:
    label_indices = np.where(labels == label)[0]
    train_idx, valid_idx = train_test_split(label_indices, test_size=0.3, random_state=42)
    train_indices.extend(train_idx)
    valid_indices.extend(valid_idx)

# Subsets for train and valid
train_dataset = Subset(dataset, train_indices)
valid_dataset = Subset(dataset, valid_indices)

# DataLoader creation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

Found 80 CSV files.


In [8]:
from torch.utils.data import DataLoader, Subset

# Ensure the 'model' folder exists
os.makedirs("model", exist_ok=True)

# Training loop for parameter grid
for params in parameter_grid:
    print(f"Training with parameters: {params}")

    # Unpack parameters
    lr = params['learning_rate']
    batch_size = params['batch_size']
    optimizer_name = params['optimizer']
    num_epochs = params['epochs']

    # Reinitialize DataLoader with the new batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize model
    model = FaceLandmarksModelAttention(input_dim=1434, hidden_size=128, num_layers=2, num_classes=4).to(device)
    # model = FaceLandmarksModel(input_size=1404, hidden_size=128, num_layers=2, num_classes=4).to(device)
    criterion = nn.CrossEntropyLoss()

    # Select optimizer
    if optimizer_name == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

    # Early stopping variables
    patience = 10  # Number of epochs with no improvement after which training stops
    best_valid_loss = float('inf')
    epochs_no_improve = 0

    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        valid_loss = 0.0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")

        # Check for improvement
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            epochs_no_improve = 0
            print(f"Validation loss improved to {best_valid_loss:.4f}")
        else:
            epochs_no_improve += 1

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    # Save the model with final parameters
    model_filename = (f"model/loss_{best_valid_loss:.4f}_lr_{lr}_batch_{batch_size}_"
                      f"opt_{optimizer_name}.pt")
    torch.save(model.state_dict(), model_filename)
    print(f"Training completed. Final model saved as {model_filename}")

Training with parameters: {'batch_size': 32, 'epochs': 200, 'learning_rate': 0.0001, 'optimizer': 'Adam'}
Epoch [1/200], Train Loss: 1.4047, Valid Loss: 1.3648
Validation loss improved to 1.3648
Epoch [2/200], Train Loss: 1.3698, Valid Loss: 1.3476
Validation loss improved to 1.3476
Epoch [3/200], Train Loss: 1.3318, Valid Loss: 1.3314
Validation loss improved to 1.3314
Epoch [4/200], Train Loss: 1.3220, Valid Loss: 1.3170
Validation loss improved to 1.3170
Epoch [5/200], Train Loss: 1.2994, Valid Loss: 1.3048
Validation loss improved to 1.3048
Epoch [6/200], Train Loss: 1.2764, Valid Loss: 1.2935
Validation loss improved to 1.2935
Epoch [7/200], Train Loss: 1.2581, Valid Loss: 1.2803
Validation loss improved to 1.2803
Epoch [8/200], Train Loss: 1.2705, Valid Loss: 1.2673
Validation loss improved to 1.2673
Epoch [9/200], Train Loss: 1.2330, Valid Loss: 1.2540
Validation loss improved to 1.2540
Epoch [10/200], Train Loss: 1.2400, Valid Loss: 1.2403
Validation loss improved to 1.2403
Epo